In [1]:
import BioSimSpace.Sandpit.Exscientia as BSS
import numpy as np
import pandas as pd

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
pdb = '6rna'

In [14]:
system = BSS.IO.readMolecules(f"{pdb}/solvated/*")

We will need to run a minimization, equilibration and production protocol

## Minimization

In [15]:
protocol = BSS.Protocol.Minimisation(steps=20000)
process = BSS.Process.Gromacs(system, protocol, work_dir=f'{pdb}/minimisation')

In [16]:
process.start()

BioSimSpace.Process.Gromacs(<BioSimSpace.System: nMolecules=12438>, BioSimSpace.Protocol.Minimisation(steps=20000, restraint=None, force_constant=10.0 kcal_per_mol/angstrom**2), exe='/home/lex/miniforge3/envs/bss/bin.AVX2_256/gmx_mpi', name='gromacs', work_dir='/home/lex/Research/kinase_intermediate_state/abfe/6rna/minimisation', seed=None)

In [17]:
minimised = process.getSystem(block=True)

In [21]:
BSS.IO.saveMolecules(f'{pdb}/minimisation/final/final', minimised, ['gro87', 'grotop'])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/minimisation/final/final.gro',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/minimisation/final/final.top']

In [22]:
minimised = BSS.IO.readMolecules(f'{pdb}/minimisation/final/*', make_whole=True)

In [23]:
BSS.Notebook.View(minimised).system()

NGLWidget(gui_style='ngl')

## Heating

In [25]:
heat_prot = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.Time.femtosecond, 
                                       runtime = 1*BSS.Units.Time.nanosecond,
                                       temperature_start = 100*BSS.Units.Temperature.kelvin, 
                                       temperature_end = 300*BSS.Units.Temperature.kelvin, 
                                       restraint='backbone')
heat_proc = BSS.Process.Gromacs(minimised, heat_prot, work_dir=f'{pdb}/heating')
                                       

In [26]:
heat_proc.start()

BioSimSpace.Process.Gromacs(<BioSimSpace.System: nMolecules=12438>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=1.0000 ns, temperature_start=100.0000 K, temperature_end=300.0000 K, pressure=None, tau_t=1.0000 ps, report_interval=200, restart_interval=1000, restart=False, restraint=backbone, force_constant=10.0 kcal_per_mol/angstrom**2), exe='/home/lex/miniforge3/envs/bss/bin.AVX2_256/gmx_mpi', name='gromacs', work_dir='/home/lex/Research/kinase_intermediate_state/abfe/6rna/heating', seed=None)

In [27]:
heated = heat_proc.getSystem(block=True)

In [28]:
BSS.IO.saveMolecules(f'{pdb}/heating/final/final', heated, ['gro87', 'grotop'])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/heating/final/final.gro',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/heating/final/final.top']

In [29]:
heated = BSS.IO.readMolecules(f'{pdb}/heating/final/*', make_whole=True)

In [31]:
BSS.Notebook.View(heated).system()

NGLWidget(gui_style='ngl')

## Equilibration - with restraints


In [32]:
equil_res_prot = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.Time.femtosecond, 
                                            runtime=0.5*BSS.Units.Time.nanosecond, 
                                            temperature=300*BSS.Units.Temperature.kelvin, 
                                            pressure=1*BSS.Units.Pressure.atm, 
                                            restraint='backbone')
equil_res_proc = BSS.Process.Gromacs(heated, equil_res_prot, work_dir=f'{pdb}/equil_res')

In [33]:
equil_res_proc.start()
equil_res = equil_res_proc.getSystem(block=True) 

In [34]:
BSS.IO.saveMolecules(f'{pdb}/equil_res/final/final', equil_res, ['gro87', 'grotop'])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil_res/final/final.gro',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil_res/final/final.top']

In [35]:
equil_res = BSS.IO.readMolecules(f'{pdb}/equil_res/final/*', make_whole=True)
BSS.Notebook.View(equil_res).system()

NGLWidget(gui_style='ngl')

## Equilibration - no restraints


In [36]:
equil_prot = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.Time.femtosecond, 
                                            runtime=0.2*BSS.Units.Time.nanosecond, 
                                            temperature=300*BSS.Units.Temperature.kelvin, 
                                            pressure=1*BSS.Units.Pressure.atm, 
                                            restraint=None)
equil_proc = BSS.Process.Gromacs(equil_res, equil_prot, work_dir=f'{pdb}/equil')

In [37]:
equil_proc.start()
equil = equil_proc.getSystem(block=True) 
BSS.IO.saveMolecules(f'{pdb}/equil/final/final', equil, ['gro87', 'grotop'])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil/final/final.gro',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil/final/final.top']

In [38]:
equil = BSS.IO.readMolecules(f'{pdb}/equil/final/*', make_whole=True)
BSS.Notebook.View(equil).system()

NGLWidget(gui_style='ngl')

In [40]:
traj = equil_proc.getTrajectory()

/home/lex/miniforge3/envs/bss/lib/python3.10/site-packages/BioSimSpace/Sandpit/Exscientia/Trajectory/_trajectory.py:635: UserWarning: MDAnalysis failed to read: traj=/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil/gromacs.xtc, top=/home/lex/Research/kinase_intermediate_state/abfe/6rna/equil/gromacs.tpr
  _warnings.warn(


In [56]:
import mdtraj as md
import nglview as nv

In [49]:
traj = md.load(f'{pdb}/equil/gromacs.xtc', top=f'{pdb}/equil/gromacs.gro')

In [52]:
traj = traj.image_molecules()

In [57]:
nv.show_mdtraj(traj)

NGLWidget(max_frame=100)

## Production 5 ns

In [58]:
prod_prot = BSS.Protocol.Production(timestep = 2*BSS.Units.Time.femtosecond, 
                                            runtime=5*BSS.Units.Time.nanosecond, 
                                            temperature=300*BSS.Units.Temperature.kelvin, 
                                            pressure=None, 
                                            restraint=None)
prod_proc = BSS.Process.Gromacs(equil, prod_prot, work_dir=f'{pdb}/production')

In [59]:
prod_proc.start()
prod = prod_proc.getSystem(block=True) 
BSS.IO.saveMolecules(f'{pdb}/production/final/final', prod, ['gro87', 'grotop'])

['/home/lex/Research/kinase_intermediate_state/abfe/6rna/production/final/final.gro',
 '/home/lex/Research/kinase_intermediate_state/abfe/6rna/production/final/final.top']

In [63]:
prod_traj = md.load(f'{pdb}/production/gromacs.xtc', top=f'{pdb}/production/gromacs.gro')


In [67]:
prod_traj = prod_traj.image_molecules()


In [68]:
nv.show_mdtraj(prod_traj)

NGLWidget(max_frame=2500)